## Описание тг бота для работы с warehouse-service

### Общая схема работы:
warehouse-bot/\
├── pom.xml\
└── src/\
    ├── main/\
    │   ├── java/\
    │   │   └── com/\
    │   │       └── warehouse/\
    │   │           └── bot/\
    │   │               ├── config/\
    │   │               ├── controller/\
    │   │               ├── model/\
    │   │               ├── service/\
    │   │               └── handler/\
    │   └── resources/\
    └── test/\
        └── java/\
            └── com/\
                └── warehouse/\

#### Описание
# Расширенная техническая документация: Архитектура бота для склада в Telegram

## 🏗️ **Обзор проекта**
Бот для Telegram на базе Spring Boot, предоставляющий комплексный интерфейс для управления складом посредством интеграции REST API со складской службой.

## 📁 **Подробное описание структуры проекта**

### **1. Корневые файлы конфигурации**

#### **`pom.xml`** — объектная модель проекта Maven
**Назначение**: Центральная зависимость и конфигурация сборки
**Ключевые разделы**:
```xml
<parent>
<groupId>org.springframework.boot</groupId>
<artifactId>spring-boot-starter-parent</artifactId>
<version>2.7.0</version>
</parent>
```
**Управляемые зависимости**:
- **Spring Boot Starter Web**: Возможности REST-клиента через `RestTemplate`
- **TelegramBots**: Интеграция API Telegram Bot (v6.5.0)
- **Lombok**: Генерация кода на основе аннотаций (геттеры/сеттеры/конструкторы)
- **Spring Boot Starter Test**: Поддержка фреймворка тестирования

**Плагины сборки**:
- `spring-boot-maven-plugin`: Включает создание исполняемого JAR-файла и специфичные для Spring Boot функции сборки

---

### **2. Точка входа в основное приложение**

#### **`WarehouseBotApplication.java`** - Spring Bootstrap
**Расположение**: `src/main/java/com/warehouse/bot/`
**Аннотации**:
- `@SpringBootApplication`: Объединение метааннотаций:
- `@Configuration`: Класс объявляет один или несколько методов `@Bean`
- `@ComponentScan`: Включает сканирование компонентов в текущем пакете
- `@EnableAutoConfiguration`: Включает автоматическую настройку Spring Boot

**Техническая роль**:
- Bootstraps Spring ApplicationContext
- Запускает автоматическую настройку и сканирование компонентов
- Запускает встроенный сервер Tomcat (порт 8081)

---

### **3. Уровень конфигурации (`config/`)**

#### **`BotConfig.java`** - Компонент свойств конфигурации
**Аннотации**:
- `@Configuration`: Указывает, что класс содержит определения компонента
- `@Data`: Аннотации Lombok, генерирующие геттеры/сеттеры

**Внедряемые свойства**:
```java
@Value("${telegram.bot.token}") private String botToken;
@Value("${telegram.bot.username}") private String botUsername;
@Value("${warehouse.service.url}") private String inventoryServiceUrl;
```

**Техническое назначение**:
- Централизация внешнего управления конфигурацией
- Обеспечивает типобезопасный доступ к application.properties
- Обеспечивает настройку специфичных для среды конфигураций

---

### **4. Уровень контроллера (`controller/`)**

#### **`WarehouseController.java`** - Конфигурация REST-клиента
**Аннотации**:
- `@Configuration`: Класс конфигурации для определений bean-компонентов
- `@Bean`: Фабричный метод для экземпляра RestTemplate

**Техническая реализация**:
```java
@Bean
public RestTemplate restTemplate() {
return new RestTemplate();
}
```

**Роль в архитектуре**:
- Предоставляет синглтон-bean-компонент `RestTemplate` для HTTP-операций
- Обеспечивает внедрение зависимостей HTTP-клиента во всё приложение
- Настраивает параметры пула соединений и тайм-аута

---

### **5. Уровень модели (`model/`) - Объекты передачи данных**

#### **`Product.java`** - Универсальное представление товара
**Поля**:
- `Long id`: Основной идентификатор
- `String name, category`: Классификация товара
- `Double price`: Информация о цене
- `Integer quantity`: Уровень запасов
- `Boolean isActive`: Флаг статуса товара
- `String sku, pathToPhoto`: Дополнительные метаданные товара

**Использование**: Сопоставляется с ответами API сервиса склада

#### **`Thermocup.java`** - Специализированный тип товара
**Расширенные поля**:
- `Integer category_id, inventory_id`: Реляционные идентификаторы
- `Double base_price`: Базовая цена
- `Integer starting_quantity`: Начальный запас
- `String SKU, path_to_photo`: Идентификация товара
- `Attribute Атрибуты: Вложенные свойства, специфичные для термокружки

**Сериализация JSON**: Использует `@JsonInclude(JsonInclude.Include.NON_NULL)` для разреженных обновлений

#### **`Attribute.java`** - Характеристики термокружки
**Физические свойства**:
- `Integer volume_ml`: Характеристики ёмкости
- `String color, brand, model, material`: Атрибуты продукта
- `Boolean is_hermetic`: Герметичность

**Шаблон проектирования**: Композитный шаблон для вариаций продукта

---

### **6. Сервисный уровень (`service/`) - Бизнес-логика**

#### **`WarehouseApiService.java`** - Клиент REST API
**Технические компоненты**:
- **Методы HTTP**: Реализации GET, POST, PUT, PATCH
- **Обработка ошибок**: Комплексный try-catch с логированием
- **Создание запросов**: UriComponentsBuilder для параметризованных URL
- **Обработка ответов**: Типобезопасное сопоставление ответов с использованием ParameterizedTypeReference

**Сигнатуры ключевых методов**:
```java
public List<Product> getProducts(Map<String, String> filters)
public String createThermocup(Thermocup thermocup)
public String updateReservedQuantity(Long productId, Integer quantityChange)
```

**Операции HTTP**:
- **GET /products**: Фильтруемый список товаров
- **GET /products/{id}**: Получение товара по ID
- **POST /products/thermocups/create**: Создание термокружки
- **PUT /products/thermocups/update/{id}**: Полное обновление термокружки
- **PATCH endpoints**: Частичные обновления количества товара на складе/зарезервированного товара

#### **`TelegramBotService.java`** — Реализация бота Telegram
**Наследование**: Расширяет `TelegramLongPollingBot`
**Интеграция со Spring**: Аннотация `@Component` для управления Spring

**Основные методы**:
- `onUpdateReceived(Update update)`: Основной хук обработки сообщений
- `sendMessage(Long chatId, String text)`: Доставка ответа
- `getBotUsername()`: Идентификация бота для API Telegram

**Поток сообщений**:
1. Получает объект Update из веб-хука Telegram
2. Извлекает текст сообщения и идентификатор чата
3. Делегирует обработку CommandHandler
4. Отправляет форматированный ответ обратно пользователю

---

### **7. Уровень обработчика (`handler/`) - Обработка команд**

#### **`CommandHandler.java`** - Обработчик команд с сохранением состояния
**Архитектурный шаблон**: Конечный автомат для многоэтапных диалогов

**Ключевые компоненты**:

**1. Управление состояниями**:
```java
private Map<Long, String> userStates = new HashMap<>();
```
- Отслеживает состояние диалога пользователя по идентификатору чата
- Состояния: `AWAITING_PRODUCT_ID`, `AWAITING_THERMOCUP_CREATE` и т. д.

**2. Маршрутизация команд**:
```java
public String handleCommand(String message, Long chatId)
```
- Главный диспетчер команд
- Направляет к соответствующему обработчику в зависимости от введённого текста

**3. Анализ входных данных**:
- Формат, разделённый вертикальной чертой (`|`), для сложных данных
- Преобразование типов и проверка
- Обработка ошибок при некорректном вводе

**4. Форматирование ответа**:
- Расширенное форматирование текста с эмодзи
- Табличное представление данных
- Стандартизация сообщений об ошибках

**Поддерживаемый поток команд**:
```
Пользовательский ввод → handleCommand() → Управление состоянием → Вызов API → Форматирование ответа → Пользователь
```

---

## 🔄 **Архитектура потока данных**

### **Конвейер обработки запросов**:
1. **Обновление Telegram** → `TelegramBotService.onUpdateReceived()`
2. **Извлечение команд** → `CommandHandler.handleCommand()`
3. **Проверка состояния** → Проверка карты `userStates`
4. **Интеграция API** → HTTP-вызовы `WarehouseApiService`
5. **Сопоставление ответов** → Объект Java в сообщение Telegram
6. **Доставка сообщения** → `TelegramBotService.sendMessage()`

### **Стратегия обработки ошибок**:
- **Сетевые ошибки**: Регистрируйте и возвращайте понятные пользователю сообщения
- **Ошибки проверки**: Руководство по формату входных данных
- **Ошибки API**: Сообщения об ошибках службы пересылки хранилища
- **Ошибки состояния**: Сброс диалога с инструкциями

---

## 🛡️ **Вопросы безопасности**

### **Безопасность конфигурации**:
- Токен бота вынесен в файл свойств
- Отсутствие жёстко заданных учётных данных
- Поддержка конфигурации, специфичной для среды

### **Валидация входных данных**:
- Преобразование типов с обработкой исключений
- Предотвращение SQL-инъекций через параметризованные API
- Ограничения на размер сообщений, устанавливаемые API Telegram

---

## 📊 **Мониторинг и Ведение журнала**

### **Стратегия ведения журнала**:
- SLF4J с аннотацией Lombok `@Slf4j`
- Уровень отладки для разработки
- Структурированное ведение журнала запросов/ответов API
- Отслеживание ошибок с помощью трассировки стека

Эта архитектура обеспечивает масштабируемую и удобную в обслуживании основу для управления складом через Telegram с четким разделением задач и комплексной обработкой ошибок.